In [1]:
## pip install moviepy==1.0.0

import os
import librosa
import pandas as pd
from moviepy.editor import *
from tqdm import tqdm_notebook
import numpy as np
import re

In [2]:
#from utils import make_dir
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [3]:
background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성

In [4]:
audio_label_clip_dir = os.path.join('.', 'data', 'audio_label_clip')
make_dir(audio_label_clip_dir)

In [5]:
def save(audios, label):
    data = pd.DataFrame({
        'audios':audios, 'label':label })
    data.to_csv(audio_label_clip_dir+'/audio_label_clip.csv', index=False, encoding='euc-kr')

In [8]:
## 레이블링 데이터 위치 : ./data/label
files = []
for file in os.listdir(os.path.join('.', 'data', 'label')):
    _, file_id = os.path.split(file)
    if file_id.endswith('txt'):
        name = file_id.split(".")[0]
        if(name not in files):
            files.append(name)
print(files)

background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)


['7EoOqdul6Tk_0', '7EoOqdul6Tk_1', '7EoOqdul6Tk_10', '7EoOqdul6Tk_11', '7EoOqdul6Tk_12', '7EoOqdul6Tk_13', '7EoOqdul6Tk_14', '7EoOqdul6Tk_15', '7EoOqdul6Tk_16', '7EoOqdul6Tk_17', '7EoOqdul6Tk_18', '7EoOqdul6Tk_19', '7EoOqdul6Tk_2', '7EoOqdul6Tk_20', '7EoOqdul6Tk_21', '7EoOqdul6Tk_22', '7EoOqdul6Tk_23', '7EoOqdul6Tk_24', '7EoOqdul6Tk_25', '7EoOqdul6Tk_3', '7EoOqdul6Tk_4', '7EoOqdul6Tk_5', '7EoOqdul6Tk_6', '7EoOqdul6Tk_7', '7EoOqdul6Tk_8', '7EoOqdul6Tk_9']


In [6]:
background_dir = './data/audio_label_clip_for_test/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip_for_test/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip_for_test/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)

In [9]:
audio = []
label = []
#라벨링
shi = 0
jon = 0
sha = 0
byoung = 0
sang = 0 
do = 0
mi = 0
gae = 0
background = 0
negative = 0
count = 0

# 이제 욕 추가될때마다 여기 counter 디렉터리에 추가해주면 됨
counter = {
    10: shi,
    11: jon,
    12: sha,
    13: byoung,
    14: do,
    15: sang,
    16: mi,
    17: gae,
    9: background,
    0: negative
}

for file in files:
    print("file name:" + file)
    text = './data/label/' + file + ".txt"
    aud = './data/label/' + file + ".wav"
    # 정규표현식 패턴
    pattern = r"^(.*?)_\d+$"
    # 정규표현식 매칭
    match = re.match(pattern, file)
    raw_aud = './data/audio/' + str(match.group(1)) + ".wav"

    print("read new file")
    data = pd.read_csv(text, sep="\t", names=['start', 'end', 'label'])
    print(data)
    for i in range(len(data)):
        lab = data.label[i]
        print(data.start[i], data.end[i], data.label[i])
        try:
            start = float(data.start[i])
            end = float(data.end[i])
        except ValueError:
            continue
        if start >= end:
            continue
        if np.isnan(lab):
            continue
        full_audio = AudioFileClip(raw_aud)

        # 각 라벨 이름 정의
        lab_name = {
            10: "shi",
            11: "jon",
            12: "sha",
            13: "byoung",
            14: "do",
            15: "sang",
            16: "mi",
            17: "gae",
            9: "background",
            0: "negative"
        }[lab]
        
        lab_counter = counter[lab]
        name = f"{lab_name}_{lab_counter}"
        counter[lab] += 1

        tmp_audio = full_audio.subclip(start, end)
        
        #라벨에 맞게 디렉터리 경로 초기화해주고
        if lab == 10 or lab == 11 or lab == 12 or lab == 13 or lab == 14 or lab == 15 or lab == 16 or lab == 17:
            directory = activate_dir
        elif lab == 9:
            directory = background_dir
        elif lab == 0:
            directory = negative_dir
        else:
            continue
        # 라벨에 맞는 디렉터리에 저장
        name = os.path.join(directory, name + ".wav")
        try:
            tmp_audio.write_audiofile(name)
        except OSError:
            print('oserror')
            continue
            
        audio.append(name)
        label.append(int(lab))
        print('완료됬습니다.')
    count += 1

save(audio, label)

file name:7EoOqdul6Tk_0
read new file
   start    end  label
0    0.0  2.159      0
0.0 2.159 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_0.wav
MoviePy - Done.
완료됬습니다.
file name:7EoOqdul6Tk_1
read new file
   start    end  label
0  2.159  3.959      0


2.159 3.959 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_1.wav
MoviePy - Done.
완료됬습니다.
file name:7EoOqdul6Tk_10
read new file


    start     end  label
0  20.939  22.619      0
20.939 22.619 0


...

MoviePy - Done.
완료됬습니다.
file name:7EoOqdul6Tk_8
read new file
    start    end  label
0  17.039  18.84      0
17.039 18.84 0


chunk:   0%|          | 0/40 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_21.wav


MoviePy - Done.
완료됬습니다.
file name:7EoOqdul6Tk_9
read new file
   start     end  label
0  18.84  20.939      0
18.84 20.939 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_22.wav
MoviePy - Done.
완료됬습니다.
